<a href="https://colab.research.google.com/github/hemzadev/shared-notebooks/blob/main/idcardsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: Fine-Tuning YOLO for ID Card and Field Detection
# Description: This notebook fine-tunes a YOLO model to detect ID cards and their fields.

# =============================================
# 1. Setup and Installation
# =============================================

In [4]:
# Install required libraries
!pip install ultralytics
!pip install matplotlib
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.6/932.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [5]:
# Import libraries
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
from google.colab import files
from IPython.display import Image, display

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# =============================================
# 2. Prepare Dataset
# =============================================

In [ ]:
# Upload your dataset.zip file to Google Colab
# (Use the file uploader or Google Drive)
uploaded = files.upload()

In [5]:
# Unzip the dataset
!unzip dataset.zip -d /content/

Archive:  dataset.zip
   creating: /content/dataset/
  inflating: /content/dataset/classes.txt  
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/images/
   creating: /content/dataset/images/test/
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_100.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_82.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_83.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_84.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_85.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_86.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_87.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_88.png  
  inflating: /content/dataset/images/test/augmented_id_card_template2_females_89.pn

In [9]:
# Verify the dataset structure
dataset_path = "/content/dataset"
print("Dataset structure:")

# Function to print directory structure
def print_directory_structure(path, indent=""):
    if os.path.isdir(path):
        print(f"{indent}{os.path.basename(path)}/")
        indent += "    "
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            print_directory_structure(item_path, indent)
    else:
        print(f"{indent}{os.path.basename(path)}")

# Print the structure
print_directory_structure(dataset_path)

Dataset structure:
dataset/
    labels/
        val/
            id_card_template2_males_99.txt
            id_card_template2_females_75.txt
            id_card_template2_females_71.txt
            augmented_id_card_template2_females_70.txt
            augmented_id_card_template2_females_73.txt
            id_card_template2_males_81.txt
            augmented_id_card_template2_females_99.txt
            id_card_template2_females_99.txt
            augmented_id_card_template2_males_74.txt
            augmented_id_card_template2_females_71.txt
            id_card_template2_males_69.txt
            augmented_id_card_template2_males_81.txt
            augmented_id_card_template2_males_75.txt
            augmented_id_card_template2_females_65.txt
            id_card_template2_females_67.txt
            id_card_template2_males_70.txt
            augmented_id_card_template2_females_64.txt
            augmented_id_card_template2_males_73.txt
            id_card_template2_males_76.txt
          

# =============================================
# 3. Load and Inspect the YOLO Model
# =============================================


In [10]:
# Load a pre-trained YOLO model (e.g., YOLOv8n)
model = YOLO('yolov8n.pt')  # You can use 'yolov8s', 'yolov8m', etc.

100%|██████████| 6.25M/6.25M [00:00<00:00, 314MB/s]


In [11]:
# Inspect the model architecture
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

# =============================================
# 4. Train the Model
# =============================================

In [2]:
# Define training parameters
epochs = 50
imgsz = 640
batch = 16
data_yaml = "/content/dataset/data.yaml"

In [3]:
# Start training
results = model.train(data=data_yaml, epochs=epochs, imgsz=imgsz, batch=batch)

NameError: name 'model' is not defined

# =============================================
# 5. Evaluate the Model
# =============================================

In [ ]:
# Validate the model on the validation set
metrics = model.val()  # This will generate metrics like mAP, precision, recall, etc.
print(metrics)

In [ ]:
# Plot training metrics
results_path = "/content/runs/detect/train"  # Path to training results
metrics_file = os.path.join(results_path, "results.png")

if os.path.exists(metrics_file):
    print("Training Metrics:")
    display(Image(filename=metrics_file))
else:
    print("Training metrics plot not found.")

# =============================================
# 6. Visualize Predictions
# =============================================

In [ ]:
# Test the model on a few validation images
val_images_path = "/content/dataset/images/val"
val_images = os.listdir(val_images_path)[:5]  # Select first 5 images for visualization

for image_name in val_images:
    image_path = os.path.join(val_images_path, image_name)
    results = model.predict(source=image_path, save=True, conf=0.5)  # Adjust confidence threshold
    predicted_image_path = f"/content/runs/detect/predict/{image_name}"
    if os.path.exists(predicted_image_path):
        print(f"Predictions for {image_name}:")
        display(Image(filename=predicted_image_path))
    else:
        print(f"No predictions found for {image_name}.")

# =============================================
# 7. Save and Export the Model
# =============================================

In [ ]:
# Save the fine-tuned model
model.save("/content/fine_tuned_yolo.pt")

# Export the model to ONNX format (optional)
model.export(format="onnx")

# =============================================
# 8. Additional Visualizations (Optional)
# =============================================

In [ ]:
# Plot confusion matrix
confusion_matrix_path = os.path.join(results_path, "confusion_matrix.png")
if os.path.exists(confusion_matrix_path):
    print("Confusion Matrix:")
    display(Image(filename=confusion_matrix_path))
else:
    print("Confusion matrix not found.")

# Plot precision-recall curve
pr_curve_path = os.path.join(results_path, "PR_curve.png")
if os.path.exists(pr_curve_path):
    print("Precision-Recall Curve:")
    display(Image(filename=pr_curve_path))
else:
    print("Precision-recall curve not found.")

# =============================================
# 9. Cleanup (Optional)
# =============================================

In [ ]:
# Remove unnecessary files to free up space
!rm -rf /content/runs/detect/predict  # Remove prediction results
!rm -rf /content/dataset  # Remove dataset (if no longer needed)

# =============================================
# End of Notebook
# =============================================